In [212]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
import copy
import cmath as cm


In [2]:
def PrettyPrintBinary(myState, do_return=False):
    to_print = "{ "
    vec = StateToVec(myState)
    length = vec.shape[0]
    bin_len = int(m.log2(length))
    not_first = False
    for i in range(length):
        if vec[i] == 0+0.j:
            continue
        if not_first:
            to_print += " + "
        to_print += f"{vec[i]} |{bin(i)[2:].zfill(bin_len)}>"
        not_first = True
    to_print += "}"
    if do_return: return to_print
    print(to_print)

In [3]:
def PrettyPrintInteger(myState, do_return=False):
    to_print = "{ "
    vec = StateToVec(myState)
    length = vec.shape[0]
    bin_len = int(m.log2(length))
    not_first = False
    for i in range(length):
        if vec[i] == 0+0.j:
            continue
        if not_first:
            to_print += " + "
        to_print += f"{vec[i]} |{i}>"
        not_first = True
    to_print += "}"
    if do_return: return to_print
    print(to_print)

In [8]:
myState2=[
   (np.sqrt(0.1)*1.j, '101'),
   (np.sqrt(0.5), '000'),
   (-np.sqrt(0.4), '010' )]
PrettyPrintBinary(myState2)
PrettyPrintInteger(myState2)

{ (0.7071067811865476+0j) |000> + (-0.6324555320336759+0j) |010> + 0.31622776601683794j |101>}
{ (0.7071067811865476+0j) |0> + (-0.6324555320336759+0j) |2> + 0.31622776601683794j |5>}


In [142]:
def StateToVec(myState):
    vec_len = 2**len(myState[0][1])
    to_return = np.zeros(vec_len, dtype = "complex_")
    for s in myState:
        idx = int(s[1], 2)
        to_return[idx] += s[0]
    return to_return

In [143]:
print(StateToVec(myState2))

[ 0.70710678+0.j          0.        +0.j         -0.63245553+0.j
  0.        +0.j          0.        +0.j          0.        +0.31622777j
  0.        +0.j          0.        +0.j        ]


In [140]:
def VecToState(myState):
    to_return = []
    length = myState.shape[0]
    bin_len = int(m.log2(length))
    for i in range(length):
        if myState[i] == 0 + 0.j:
            continue
        to_return.append([myState[i], bin(i)[2:].zfill(bin_len)])
    return to_return

In [145]:
PrettyPrintBinary(VecToState(StateToVec(myState2)))

{ (0.7071067811865476+0j) |000> + (-0.6324555320336759+0j) |010> + 0.31622776601683794j |101>}


In [11]:
def add_duplicates(state):
    return VecToState(StateToVec(state))

In [12]:
myState3=[
  (-np.sqrt(0.125), '11'),
  (np.sqrt(0.1), '00'),
  (np.sqrt(0.4), '01'),
  (-np.sqrt(0.125), '11')
]

In [15]:
PrettyPrintBinary(add_duplicates(myState3))

{ (0.31622776601683794+0j) |00> + (0.6324555320336759+0j) |01> + (-0.7071067811865476+0j) |11>}


In [267]:
class Quantum_Computer():
    
    def __init__(self, description, from_file=False):
        if from_file:
            with open(description) as f:
                lines = f.read().splitlines(False)
        else:
            lines = description.splitlines(False)
        self.n = int(lines[0])
        self.gates = []
        for line in lines[1:]:
            pcs = line.split(' ')
            print(pcs)
            if pcs[0] == 'H':
                self.gates.append((0, int(pcs[1])))
            elif pcs[0] == 'P':
                self.gates.append((1, int(pcs[1]), float(pcs[2])))
            elif pcs[0] == 'CNOT':
                self.gates.append((2, int(pcs[1]), int(pcs[2])))
            elif pcs[0] == "MEASURE":
                self.gates.append((3,0))
        return
    
    def H(self, wire, state):
        new_state = []
        for element in state:
            new_state = new_state + self.H_element(wire, element)
        return add_duplicates(new_state)
    
    def H_element(self, wire, element):
        alt_element = copy.deepcopy(element)
        alt_element[1] = alt_element[1][:wire] + str(self.flip_bit(alt_element[1][wire])) + alt_element[1][wire+1:]
        alt_element[0] = alt_element[0] * (1/m.sqrt(2))
        element[0] = element[0] * (1/m.sqrt(2))
        if element[1][wire] == '1': element[0] = -element[0]
        return [element, alt_element]
        
    def phase(self, wire, phase, state):
        [self.phase_element(wire, phase, element) for element in state]
        return add_duplicates(state)
    
    def phase_element(self, wire, phase, element):
        if element[1][wire] == '0':
            return element
        else:
            element[0] = element[0] * cm.exp(phase*1.j)
            return element
    
    def CNOT(self, control, change, state):
        [self.CNOT_element(control, change, element) for element in state]
        return add_duplicates(state)
        
    def CNOT_element(self, control, change, element):    
        if element[1][control] == '0':
            return element
        else:
            element[1] = element[1][:change] + str(self.flip_bit(element[1][change])) + element[1][change+1:]
            return element
    
    def measure(self, state, size=1):
        states = []
        probabilities = []
        for element in state:
            states.append(element[1])
            probabilities.append(element[0].real)
        return np.random.choice(states, size=size, p=probabilities)
    
    def run(self, initial_state=0):
        if initial_state == 0:
            vec = np.zeros(2**self.n, dtype = "complex_")
            vec[0] = complex(1)
            initial_state = VecToState(vec)
        state = initial_state
        for gate in self.gates:
            state = self.run_gate(gate, state)
        return state
            
    def run_gate(self, gate, state):
        if gate[0] == 0:
            state = self.H(gate[1], state)
        elif gate[0] == 1:
            state = self.phase(gate[1], gate[2], state)
        elif gate[0] == 2:
            state = self.CNOT(gate[1], gate[2], state)
        elif gate[0] == 3:
            print(f"Measurement: {self.measure(state)}")
        return state
    
    def flip_bit(self, bit):
        bit = int(bit)
        if bit == 0:
            return 1
        return 0

In [240]:
test_description = """3
H 1
H 2
P 2 0.3
CNOT 2 1
H 1
H 2
CNOT 2 0
MEASURE"""

In [268]:
test_computer = Quantum_Computer(test_description)

['H', '1']
['H', '2']
['P', '2', '0.3']
['CNOT', '2', '1']
['H', '1']
['H', '2']
['CNOT', '2', '0']
['MEASURE']


In [294]:
PrettyPrintBinary(test_computer.run())

Measurement: ['101']
{ (0.9776682445628027+0.14776010333066972j) |000> + (0.022331755437197065-0.14776010333066972j) |101>}


In [271]:
test_list = [(1, 3), (2, 3)]
test_list.append([(4, 5), (7, 6)])
print(test_list)

[(1, 3), (2, 3), [(4, 5), (7, 6)]]
